In [ ]:
import numpy as np
import terra

from meerkat.contrib.eeg import build_stanford_eeg_dp

from domino.emb.eeg import generate_words_dp, embed_words, embed_eeg
from domino.evaluate import run_sdms, score_sdm_explanations, score_sdms, run_sdm
from domino.sdm import MixtureModelSDM, SpotlightSDM
from domino.slices import collect_settings
from domino.train import score_settings, synthetic_score_settings, train_settings
from domino.utils import split_dp, balance_dp

import meerkat as mk
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
setting_dp = collect_settings.out(load=True)
setting_dp

In [ ]:
run_sdms_dp = run_sdms.out(load=True)
run_sdms_dp.head()

In [ ]:
score_dp = mk.DataPanel.from_pandas(score_sdms.out(load=True))
score_dp

In [ ]:
results_dp = mk.merge(
    score_dp,
    run_sdms_dp["config/sdm", "alpha","run_sdm_run_id"], # include any other columns here you'd like to analyze
    on="run_sdm_run_id"
)
emb_col = results_dp["config/sdm"].map(lambda x: x["sdm_config"]["emb"][0])
results_dp["emb_type"] = emb_col

results_df = results_dp.to_pandas()
results_df

In [ ]:
metric = "precision_at_10"
grouped_df = results_df.iloc[results_df.reset_index().groupby(["slice_name", "slice_idx","emb_type","alpha"])[metric].idxmax().astype(int)]

#ax = sns.lineplot(data=grouped_df,x="alpha",y=metric, hue="emb_type",style="slice_name")
ax = sns.barplot(data=grouped_df,x=metric,y="emb_type")
plt.show()

In [ ]:
metric = "auroc"
grouped_df = results_df.iloc[results_df.reset_index().groupby(["slice_name", "slice_idx","emb_type","alpha"])[metric].idxmax().astype(int)]

#ax = sns.lineplot(data=grouped_df,x="alpha",y=metric, hue="emb_type",style="slice_name")
ax = sns.barplot(data=grouped_df,x=metric,y="emb_type")
plt.show()

# Explanations

In [ ]:
words_dp = embed_words.out(load=True)

# get multimodal sdm run ids
slice_id = 1
alpha = 0.9

specific_run = grouped_df[grouped_df["emb_type"]=="multimodal"][grouped_df["slice_idx"]==slice_id][grouped_df["alpha"]==alpha]
specific_run

In [ ]:
pred_slice_idx = specific_run["pred_slice_idx"].values[0]
_, expl_dp = run_sdm.out(specific_run["run_sdm_run_id"].values[0],load=True)
expl_dp[(-expl_dp["pred_slices"].data[:,pred_slice_idx]).argsort()][:10]

In [ ]:
expl_dp[(-expl_dp["pred_slices"].data[:,pred_slice_idx]).argsort()][10:20]